In [ ]:
import os
import requests
import pandas as pd
from typing import List, Optional

DATASET_ID = "p6dx-8zbt"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

meta = requests.get(META_URL, headers=HEADERS, timeout=60)
meta.raise_for_status()
cols = meta.json().get("columns", [])
df_cols = pd.DataFrame(
    [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
    columns=["name", "fieldName", "type"]
).dropna(subset=["fieldName"]).reset_index(drop=True)

columns_set = set(df_cols["fieldName"].str.lower())

def pick_field(candidates: List[str], fallback_contains: List[str] = None, prefer_contains: List[str] = None) -> Optional[str]:
    for c in candidates:
        if c and c.lower() in columns_set:
            return c
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains   or []
    matches = []
    for f in df_cols["fieldName"].str.lower():
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))
    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    best_real = df_cols.loc[df_cols["fieldName"].str.lower() == best_lower, "fieldName"].iloc[0]
    return best_real

CAMPO_DEPTO = pick_field(
    ["departamento", "departamento_ejecucion", "departamento_proceso", "departamento_entidad"],
    fallback_contains=["depar"],
    prefer_contains=["departamento", "ejec", "proceso", "entidad"]
)
CAMPO_MPIO  = pick_field(
    ["municipio", "municipio_ejecucion", "municipio_proceso", "municipio_entidad"],
    fallback_contains=["muni"],
    prefer_contains=["municipio", "ejec", "proceso", "entidad"]
)
CAMPO_TIPO  = pick_field(
    ["tipo_de_contrato", "tipo_contrato", "tipocontrato"],
    fallback_contains=["tipo", "contrat"],
    prefer_contains=["tipo", "contrato"]
)
CAMPO_OBJ   = pick_field(
    ["objeto_a_contratar", "objeto_del_proceso", "nombre_del_proceso", "objeto"],
    fallback_contains=["obj"],
    prefer_contains=["objeto", "proceso", "nombre"]
)

MUNICIPIOS_VALLE = [
    "SANTIAGO DE CALI", "CALI", "PALMIRA", "YUMBO", "JAMUNDI", "JAMUNDÍ", "CANDELARIA", "PRADERA", "FLORIDA", "VIJES", "LA CUMBRE", "DAGUA",
    "RESTREPO", "YOTOCO", "CALIMA", "EL DARIEN", "EL DARÍEN", "GUADALAJARA DE BUGA", "BUGA", "BUGALAGRANDE", "TULUA", "TULUÁ", "SEVILLA", "CAICEDONIA", "TRUJILLO", "RIOFRIO", "RIOFRÍO",
    "SAN PEDRO", "GUACARI", "GUACARÍ", "GINEBRA", "EL CERRITO", "EL DOVIO", "ROLDANILLO", "LA UNION", "LA UNIÓN", "TORO", "OBANDO", "ULLOA", "VERSALLES", "ZARZAL", "CARTAGO", "ANSERMANUEVO", "ARGELIA", "BOLIVAR", "BOLÍVAR",
    "EL CAIRO", "EL AGUILA", "EL ÁGUILA", "ANDALUCIA", "ANDALUCÍA", "BUENAVENTURA"
]

KEYWORDS_OBRAS = [
    "OBRA", "OBRAS", "CONSTRUCCI", "MANTENIM", "MEJORAM", "INFRAESTRUC", "VIAL", "PUENTE", "CARRETER", "VIA", "VÍA", "PAVIMENT", "PLACAHUELLA", "CICLOV", "TRAMO",
    "ACUEDUCTO", "ALCANTARILL", "SANEAM", "DRENAJE", "COLEGIO", "ESCUELA", "EDUCAT", "HOSPIT", "SALUD", "PARQUE", "POLIDEPORT", "BIBLIOTECA", "CENTRO CULTURAL",
    "URBANIZ", "ANDEN", "ANDÉN", "MALECON", "MALECÓN", "TERMINAL", "REHABILIT", "RESTAUR", "REPARACI", "ADECUACI", "CONSERV"
]
KEYWORDS_INTERV = ["INTERVENTOR"]

obra_clauses = []
if CAMPO_TIPO:
    obra_clauses.append(f"upper({CAMPO_TIPO}) = upper('Obra')")
if CAMPO_OBJ:
    like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
    like_interv = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_INTERV]
    interv_and_obra = "( (" + " OR ".join(like_interv) + ") AND (" + " OR ".join(like_obras) + ") )"
    obras_only = "(" + " OR ".join(like_obras) + ")"
    obra_clauses.extend([interv_and_obra, obras_only])

if not obra_clauses:
    raise ValueError("No se encontraron campos para identificar 'obras'.")

if CAMPO_DEPTO:
    geo_clause = f"upper({CAMPO_DEPTO}) = upper('Valle del Cauca')"
elif CAMPO_MPIO:
    mpio_list = ", ".join(["'" + m.replace("'", "''") + "'" for m in MUNICIPIOS_VALLE])
    geo_clause = f"upper({CAMPO_MPIO}) IN ({mpio_list})"
else:
    raise ValueError("No se encontró ni campo de Departamento ni de Municipio.")

WHERE = f"{geo_clause} AND (" + " OR ".join(obra_clauses) + ")"

assert "None" not in WHERE
assert any(tok in WHERE for tok in ("upper(", " like ", " IN "))

print("WHERE SoQL construido:", WHERE)

params_total = {"$select": "count(1)"}
res_total = requests.get(BASE_URL, params=params_total, headers=HEADERS, timeout=180)
res_total.raise_for_status()
total_dataset = int(res_total.json()[0]["count_1"]) if res_total.json() else 0

params_geo = {"$select": "count(1)", "$where": geo_clause}
res_geo = requests.get(BASE_URL, params=params_geo, headers=HEADERS, timeout=180)
res_geo.raise_for_status()
count_geo = int(res_geo.json()[0]["count_1"]) if res_geo.json() else 0

params_count = {"$select": "count(1)", "$where": WHERE}
res_count = requests.get(BASE_URL, params=params_count, headers=HEADERS, timeout=180)
res_count.raise_for_status()
count_val = int(res_count.json()[0]["count_1"]) if res_count.json() else 0

if CAMPO_OBJ:
    params_interv = {"$select": "count(1)", "$where": geo_clause + " AND (" + " OR ".join(like_interv) + ")"}
    res_interv = requests.get(BASE_URL, params=params_interv, headers=HEADERS, timeout=180)
    res_interv.raise_for_status()
    count_interv_geo = int(res_interv.json()[0]["count_1"]) if res_interv.json() else 0
else:
    count_interv_geo = None

print("===== MINI REPORTE DE COBERTURA =====")
print(f"Total dataset (todas las filas): {total_dataset:,}")
print(f"Solo geografía (Valle del Cauca): {count_geo:,}")
print(f"Valle + Obras (incluye interventoría): {count_val:,}")
if count_interv_geo is not None:
    print(f"   └─ De los geográficos, con mención a INTERVENTOR*: {count_interv_geo:,}")
print("====================================")

params_preview = {"$where": WHERE, "$limit": 100}
preview = requests.get(BASE_URL, params=params_preview, headers=HEADERS, timeout=180)
preview.raise_for_status()
df_preview = pd.DataFrame(preview.json())

pd.set_option('display.max_columns', None)
print("Filas en la vista previa:", len(df_preview))
print("Columnas en la vista previa:", len(df_preview.columns))
print("Columnas detectadas:", list(df_preview.columns))
df_preview.head(100)

assert " AND (" in WHERE and ")" in WHERE
assert isinstance(count_val, int) and count_val >= 0
if total_dataset < 1_000_000:
    print("Aviso: El total del dataset es < 1M. Verificar ID o límites de acceso.")
if count_val < 100:
    print("Aviso: El conteo final es < 100. Considerar ampliar keywords.")


WHERE SoQL construido: upper(departamento_entidad) = upper('Valle del Cauca') AND (upper(tipo_de_contrato) = upper('Obra'))
===== MINI REPORTE DE COBERTURA =====
Total dataset (todas las filas): 7,548,652
Solo geografía (Valle del Cauca): 598,536
Valle + Obras (incluye interventoría): 6,815
Filas en la vista previa: 100
Columnas en la vista previa: 57
Columnas detectadas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_2', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'fecha_de_recepcion_de', 'fecha_de_apertura_de_respuesta', 'fecha_de_apertura_efectiva', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invita

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from typing import List, Optional

# =========================
# Configuración y metadatos
# =========================
DATASET_ID = "p6dx-8zbt"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

meta = requests.get(META_URL, headers=HEADERS, timeout=60)
meta.raise_for_status()
cols = meta.json().get("columns", [])
df_cols = pd.DataFrame(
    [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
    columns=["name", "fieldName", "type"]
).dropna(subset=["fieldName"]).reset_index(drop=True)

columns_set = set(df_cols["fieldName"].str.lower())

def pick_field(candidates: List[str], fallback_contains: List[str] = None, prefer_contains: List[str] = None) -> Optional[str]:
    for c in candidates:
        if c and c.lower() in columns_set:
            return c
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains   or []
    matches = []
    for f in df_cols["fieldName"].str.lower():
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))
    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    best_real = df_cols.loc[df_cols["fieldName"].str.lower() == best_lower, "fieldName"].iloc[0]
    return best_real

CAMPO_DEPTO = pick_field(
    ["departamento", "departamento_ejecucion", "departamento_proceso", "departamento_entidad"],
    fallback_contains=["depar"],
    prefer_contains=["departamento", "ejec", "proceso", "entidad"]
)
CAMPO_MPIO  = pick_field(
    ["municipio", "municipio_ejecucion", "municipio_proceso", "municipio_entidad"],
    fallback_contains=["muni"],
    prefer_contains=["municipio", "ejec", "proceso", "entidad"]
)
CAMPO_TIPO  = pick_field(
    ["tipo_de_contrato", "tipo_contrato", "tipocontrato"],
    fallback_contains=["tipo", "contrat"],
    prefer_contains=["tipo", "contrato"]
)
CAMPO_OBJ   = pick_field(
    ["objeto_a_contratar", "objeto_del_proceso", "nombre_del_proceso", "objeto"],
    fallback_contains=["obj"],
    prefer_contains=["objeto", "proceso", "nombre"]
)

MUNICIPIOS_VALLE = [
    "SANTIAGO DE CALI", "CALI", "PALMIRA", "YUMBO", "JAMUNDI", "JAMUNDÍ", "CANDELARIA", "PRADERA", "FLORIDA", "VIJES", "LA CUMBRE", "DAGUA",
    "RESTREPO", "YOTOCO", "CALIMA", "EL DARIEN", "EL DARÍEN", "GUADALAJARA DE BUGA", "BUGA", "BUGALAGRANDE", "TULUA", "TULUÁ", "SEVILLA", "CAICEDONIA", "TRUJILLO", "RIOFRIO", "RIOFRÍO",
    "SAN PEDRO", "GUACARI", "GUACARÍ", "GINEBRA", "EL CERRITO", "EL DOVIO", "ROLDANILLO", "LA UNION", "LA UNIÓN", "TORO", "OBANDO", "ULLOA", "VERSALLES", "ZARZAL", "CARTAGO", "ANSERMANUEVO", "ARGELIA", "BOLIVAR", "BOLÍVAR",
    "EL CAIRO", "EL AGUILA", "EL ÁGUILA", "ANDALUCIA", "ANDALUCÍA", "BUENAVENTURA"
]

KEYWORDS_OBRAS = [
    "OBRA", "OBRAS", "CONSTRUCCI", "MANTENIM", "MEJORAM", "INFRAESTRUC", "VIAL", "PUENTE", "CARRETER", "VIA", "VÍA", "PAVIMENT", "PLACAHUELLA", "CICLOV", "TRAMO",
    "ACUEDUCTO", "ALCANTARILL", "SANEAM", "DRENAJE", "COLEGIO", "ESCUELA", "EDUCAT", "HOSPIT", "SALUD", "PARQUE", "POLIDEPORT", "BIBLIOTECA", "CENTRO CULTURAL",
    "URBANIZ", "ANDEN", "ANDÉN", "MALECON", "MALECÓN", "TERMINAL", "REHABILIT", "RESTAUR", "REPARACI", "ADECUACI", "CONSERV"
]
KEYWORDS_INTERV = ["INTERVENTOR"]

obra_clauses = []
if CAMPO_TIPO:
    obra_clauses.append(f"upper({CAMPO_TIPO}) = upper('Obra')")
like_obras = []
like_interv = []
if CAMPO_OBJ:
    like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
    like_interv = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_INTERV]
    interv_and_obra = "( (" + " OR ".join(like_interv) + ") AND (" + " OR ".join(like_obras) + ") )"
    obras_only = "(" + " OR ".join(like_obras) + ")"
    obra_clauses.extend([interv_and_obra, obras_only])

if not obra_clauses:
    raise ValueError("No se encontraron campos para identificar 'obras'.")

if CAMPO_DEPTO:
    geo_clause = f"upper({CAMPO_DEPTO}) = upper('Valle del Cauca')"
elif CAMPO_MPIO:
    mpio_list = ", ".join(["'" + m.upper().replace("'", "''") + "'" for m in MUNICIPIOS_VALLE])
    geo_clause = f"upper({CAMPO_MPIO}) IN ({mpio_list})"
else:
    raise ValueError("No se encontró ni campo de Departamento ni de Municipio.")

WHERE = f"{geo_clause} AND (" + " OR ".join(obra_clauses) + ")"

assert "None" not in WHERE
assert any(tok in WHERE for tok in ("upper(", " like ", " IN "))

print("WHERE SoQL construido:", WHERE)

params_total = {"$select": "count(1)"}
res_total = requests.get(BASE_URL, params=params_total, headers=HEADERS, timeout=180)
res_total.raise_for_status()
total_dataset = int(res_total.json()[0]["count_1"]) if res_total.json() else 0

params_geo = {"$select": "count(1)", "$where": geo_clause}
res_geo = requests.get(BASE_URL, params=params_geo, headers=HEADERS, timeout=180)
res_geo.raise_for_status()
count_geo = int(res_geo.json()[0]["count_1"]) if res_geo.json() else 0

params_count = {"$select": "count(1)", "$where": WHERE}
res_count = requests.get(BASE_URL, params=params_count, headers=HEADERS, timeout=180)
res_count.raise_for_status()
count_val = int(res_count.json()[0]["count_1"]) if res_count.json() else 0

count_interv_geo = None
if CAMPO_OBJ and like_interv:
    params_interv = {"$select": "count(1)", "$where": geo_clause + " AND (" + " OR ".join(like_interv) + ")"}
    res_interv = requests.get(BASE_URL, params=params_interv, headers=HEADERS, timeout=180)
    res_interv.raise_for_status()
    count_interv_geo = int(res_interv.json()[0]["count_1"]) if res_interv.json() else 0

print("===== MINI REPORTE DE COBERTURA =====")
print(f"Total dataset (todas las filas): {total_dataset:,}")
print(f"Solo geografía (Valle del Cauca): {count_geo:,}")
print(f"Valle + Obras (incluye interventoría): {count_val:,}")
if count_interv_geo is not None:
    print(f"   └─ De los geográficos, con mención a INTERVENTOR*: {count_interv_geo:,}")
print("====================================")

params_preview = {"$where": WHERE, "$limit": 100}
preview = requests.get(BASE_URL, params=params_preview, headers=HEADERS, timeout=180)
preview.raise_for_status()
df_preview = pd.DataFrame(preview.json())

pd.set_option('display.max_columns', None)
print("Filas en la vista previa:", len(df_preview))
print("Columnas en la vista previa:", len(df_preview.columns))
print("Columnas detectadas:", list(df_preview.columns))
display(df_preview.head(100))

assert " AND (" in WHERE and ")" in WHERE
assert isinstance(count_val, int) and count_val >= 0
if total_dataset < 1_000_000:
    print("Aviso: El total del dataset es < 1M. Verificar ID o límites de acceso.")
if count_val < 100:
    print("Aviso: El conteo final es < 100. Considerar ampliar keywords.")

# ============================================
# Variables relevantes para cálculos y reportes
# ============================================
VAR_FECHA_INICIO    = "fecha_adjudicacion"
VAR_DURACION        = "duración"
VAR_UNIDAD_DURACION = "unidad_de_duracion"
VAR_PRECIO_BASE     = "precio_base"
VAR_VALOR_INICIAL   = "Valor Total Adjudicación"
VAR_ESTADO          = "estado_resumen"
VAR_ORDEN_ENTIDAD   = "ordenentidad"
VAR_MODALIDAD       = "modalidad_de_contratacion"

for col in [VAR_FECHA_INICIO, VAR_DURACION, VAR_UNIDAD_DURACION, VAR_PRECIO_BASE, VAR_VALOR_INICIAL, VAR_ESTADO, VAR_ORDEN_ENTIDAD, VAR_MODALIDAD]:
    if col not in df_preview.columns:
        df_preview[col] = pd.NA

df_preview[VAR_FECHA_INICIO]  = pd.to_datetime(df_preview[VAR_FECHA_INICIO], errors="coerce")
df_preview[VAR_DURACION]      = pd.to_numeric(df_preview[VAR_DURACION], errors="coerce")
df_preview[VAR_PRECIO_BASE]   = pd.to_numeric(df_preview[VAR_PRECIO_BASE], errors="coerce")
df_preview[VAR_VALOR_INICIAL] = pd.to_numeric(df_preview[VAR_VALOR_INICIAL], errors="coerce")

u = df_preview[VAR_UNIDAD_DURACION].astype(str).str.lower().str.normalize('NFKD').str.encode('ascii', 'ignore').str.decode('ascii')
is_mes = u.str.contains("mes")
is_dia = u.str.contains("dia")
is_ano = u.str.contains("ano")

duracion_dias = np.where(is_dia, df_preview[VAR_DURACION],
                   np.where(is_ano, df_preview[VAR_DURACION] * 365,
                     np.where(is_mes, df_preview[VAR_DURACION] * 30, df_preview[VAR_DURACION] * 30)))
df_preview["duracion_dias_calc"] = duracion_dias

df_preview["fecha_fin_teorica"] = pd.to_datetime(df_preview[VAR_FECHA_INICIO], errors="coerce") + pd.to_timedelta(df_preview["duracion_dias_calc"], unit="D")

today = pd.Timestamp.today().normalize()
cond_valid = df_preview[VAR_FECHA_INICIO].notna() & pd.notna(df_preview["duracion_dias_calc"]) & df_preview["fecha_fin_teorica"].notna()
df_preview["terminada?"] = np.where(cond_valid & (today > df_preview["fecha_fin_teorica"]), "sí, validar GENTE", "en ejecución")

if "adjudicado" in df_preview.columns:
    adj = df_preview["adjudicado"].astype(str).str.strip().str.lower()
    mask_adj = adj.isin(["si", "sí"])
    df_adj = df_preview[mask_adj].copy()

    print("\n===== ESTADÍSTICA DESCRIPTIVA (adjudicado == Sí) =====")
    print("Registros adjudicados:", len(df_adj))

    if len(df_adj):
        print("\nDuración (días) - describe():")
        print(df_adj["duracion_dias_calc"].describe())

        print("\nDistribución unidad_de_duracion:")
        print(df_adj[VAR_UNIDAD_DURACION].value_counts(dropna=False).head(20))

        if df_adj[VAR_PRECIO_BASE].notna().any():
            print("\nPrecio base - suma/promedio:")
            print("Suma:", float(df_adj[VAR_PRECIO_BASE].sum()))
            print("Promedio:", float(df_adj[VAR_PRECIO_BASE].mean()))

        if df_adj[VAR_VALOR_INICIAL].notna().any():
            print("\nValor Total Adjudicación - suma/promedio:")
            print("Suma:", float(df_adj[VAR_VALOR_INICIAL].sum()))
            print("Promedio:", float(df_adj[VAR_VALOR_INICIAL].mean()))

        print("\nEstado 'terminada?':")
        print(df_adj["terminada?"].value_counts(dropna=False))

        if CAMPO_MPIO and CAMPO_MPIO in df_adj.columns:
            print("\nTop municipios por conteo (adjudicados):")
            print(df_adj[CAMPO_MPIO].value_counts(dropna=False).head(15))

        if "entidad" in df_adj.columns:
            print("\nTop entidades por conteo (adjudicados):")
            print(df_adj["entidad"].value_counts(dropna=False).head(15))

        if VAR_MODALIDAD in df_adj.columns:
            print("\nModalidad de contratación (Top 15):")
            print(df_adj[VAR_MODALIDAD].value_counts(dropna=False).head(15))

        if VAR_ORDEN_ENTIDAD in df_adj.columns:
            print("\nOrden de la entidad:")
            print(df_adj[VAR_ORDEN_ENTIDAD].value_counts(dropna=False))
    print("======================================================")
else:
    print("\nNo existe la columna 'adjudicado' en la vista previa; se omiten reportes condicionados.")


WHERE SoQL construido: upper(departamento_entidad) = upper('Valle del Cauca') AND (upper(tipo_de_contrato) = upper('Obra'))
===== MINI REPORTE DE COBERTURA =====
Total dataset (todas las filas): 7,548,652
Solo geografía (Valle del Cauca): 598,536
Valle + Obras (incluye interventoría): 6,815
Filas en la vista previa: 100
Columnas en la vista previa: 57
Columnas detectadas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_2', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'fecha_de_recepcion_de', 'fecha_de_apertura_de_respuesta', 'fecha_de_apertura_efectiva', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invita

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_2,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_3,fecha_adjudicacion,fecha_de_publicacion
0,ESCUELA DE POLICIA SIMON BOLIVAR,800141336,Valle del Cauca,No Definido,Territorial,Centralizada,CO1.REQ.599106,PN ESBOL SA MC 009 2018,701707044,CO1.BDOS.579367,MANTENIMIENTO INTEGRAL PREVENTIVO Y CORRECTIVO...,MANTENIMIENTO INTEGRAL PREVENTIVO Y CORRECTIVO...,Presentación de observaciones,2018-10-25T00:00:00.000,2018-10-25T00:00:00.000,2018-10-25T00:00:00.000,123418000,Selección Abreviada de Menor Cuantía,Presupuesto menor al 10% de la Menor Cuantía,30,día(s),2018-11-14T00:00:00.000,2018-11-14T00:00:00.000,2018-11-14T00:00:00.000,Tuluá,CONTRATOS ESBOL,0,0,210,0,0,0,0,0,0,Evaluación,60,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.72102900,Cerrado,Obra,No Definido,"V172101500, V172103300",{'url': 'https://community.secop.gov.co/Public...,701707044,Presentación de observaciones,NaN,NaN,NaN
1,CVC,890399002,Valle del Cauca,Cali,Nacional,Centralizada,CO1.REQ.6938485,CVC SABS 833 2024,702217274,CO1.BDOS.6809188,DESCOLMATACION Y LIMPIEZA PARA EL MEJORAMIENT...,AUNAR ESFUERZOS TECNICOS Y RECURSOS ECONOMICOS...,Presentación de oferta,2024-10-03T00:00:00.000,2024-10-03T00:00:00.000,NaN,103992407,Contratación Directa (con ofertas),Contratos o convenios Interadministrativos (co...,0,día(s),2024-10-08T00:00:00.000,NaN,2024-10-03T00:00:00.000,Cali,DIRECCION DE GESTION AMBIENTAL,1,1,1,0,0,0,0,1,0,Seleccionado,70,Si,CO1.AWD.2060326,705266492,Valle del Cauca,Vijes,103992407,YAMILETH CONDE POVEDA,ALCALDIA MUNICIPIO DE VIJES,800243022,V1.80101500,Abierto,Obra,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,702217274,Adjudicado,2024-10-03T00:00:00.000,2024-10-17T00:00:00.000,NaN
2,MUNICIPIO DE EL CERRITO.,800100533,Valle del Cauca,El Cerrito,Territorial,Centralizada,CO1.REQ.4139730,LP-VALLE-ELCERRITO-0002-2023,703714378,CO1.BDOS.4043628,ADECUACION Y ACONDICIONAMIENTO EL SENDERO PEAT...,ADECUACION Y ACONDICIONAMIENTO EL SENDERO PEAT...,Presentación de observaciones,2023-02-21T00:00:00.000,2023-02-21T00:00:00.000,2023-02-21T00:00:00.000,594739270,Licitación pública Obra Publica,No Defenido,90,día(s),2023-03-21T00:00:00.000,2023-03-30T00:00:00.000,2023-03-30T00:00:00.000,El Cerrito,ADQUISICION DE BIENES Y SERVICIOS 2023,0,0,115,0,0,0,0,0,0,Evaluación,60,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.72141100,Cerrado,Obra,No Definido,V172141000,{'url': 'https://community.secop.gov.co/Public...,703714378,Presentación de observaciones,NaN,NaN,NaN
3,ALCALDIA MUNICIPAL DE BUGA,891380033,Valle del Cauca,Guadalajara De Buga,Territorial,Centralizada,CO1.REQ.7002086,SAMC-SOP-1500-1658-2024 (Manifestación de inte...,704246388,CO1.BDOS.6810260,ADECUACIONES Y MANTENIMIENTO DE LAS PTAR EN LA...,REALIZAR LAS LABORES DE ADECUACIONES Y MANTENI...,Manifestación de interés (Menor Cuant


===== ESTADÍSTICA DESCRIPTIVA (adjudicado == Sí) =====
Registros adjudicados: 26

Duración (días) - describe():
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: duracion_dias_calc, dtype: float64

Distribución unidad_de_duracion:
unidad_de_duracion
día(s)     13
Mes(es)    13
Name: count, dtype: int64

Precio base - suma/promedio:
Suma: 80438428834.0
Promedio: 3093785724.3846154

Estado 'terminada?':
terminada?
en ejecución    26
Name: count, dtype: int64

Top entidades por conteo (adjudicados):
entidad
SECRETARIA DE EDUCACION DE CALI                                                                               5
CVC                                                                                                           3
DEPARTAMENTO ADMINISTRATIVO DE GESTION DEL MEDIO AMBIENTE DAGMA                                               2
POLICIA METROPOLITANA SANTIAGO DE CALI                                                       

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
from typing import List, Optional

DATASET_ID = "p6dx-8zbt"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

meta = requests.get(META_URL, headers=HEADERS, timeout=60)
meta.raise_for_status()
cols = meta.json().get("columns", [])
df_cols = pd.DataFrame(
    [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
    columns=["name", "fieldName", "type"]
).dropna(subset=["fieldName"]).reset_index(drop=True)

columns_set = set(df_cols["fieldName"].str.lower())

def pick_field(candidates: List[str], fallback_contains: List[str] = None, prefer_contains: List[str] = None) -> Optional[str]:
    for c in candidates:
        if c and c.lower() in columns_set:
            return c
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains   or []
    matches = []
    for f in df_cols["fieldName"].str.lower():
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))
    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    best_real = df_cols.loc[df_cols["fieldName"].str.lower() == best_lower, "fieldName"].iloc[0]
    return best_real

CAMPO_DEPTO = pick_field(
    ["departamento", "departamento_ejecucion", "departamento_proceso", "departamento_entidad"],
    fallback_contains=["depar"],
    prefer_contains=["departamento", "ejec", "proceso", "entidad"]
)
CAMPO_MPIO  = pick_field(
    ["municipio", "municipio_ejecucion", "municipio_proceso", "municipio_entidad"],
    fallback_contains=["muni"],
    prefer_contains=["municipio", "ejec", "proceso", "entidad"]
)
CAMPO_TIPO  = pick_field(
    ["tipo_de_contrato", "tipo_contrato", "tipocontrato"],
    fallback_contains=["tipo", "contrat"],
    prefer_contains=["tipo", "contrato"]
)
CAMPO_OBJ   = pick_field(
    ["objeto_a_contratar", "objeto_del_proceso", "nombre_del_proceso", "objeto"],
    fallback_contains=["obj"],
    prefer_contains=["objeto", "proceso", "nombre"]
)

MUNICIPIOS_VALLE = [
    "SANTIAGO DE CALI", "CALI", "PALMIRA", "YUMBO", "JAMUNDI", "JAMUNDÍ", "CANDELARIA", "PRADERA", "FLORIDA", "VIJES", "LA CUMBRE", "DAGUA",
    "RESTREPO", "YOTOCO", "CALIMA", "EL DARIEN", "EL DARÍEN", "GUADALAJARA DE BUGA", "BUGA", "BUGALAGRANDE", "TULUA", "TULUÁ", "SEVILLA", "CAICEDONIA", "TRUJILLO", "RIOFRIO", "RIOFRÍO",
    "SAN PEDRO", "GUACARI", "GUACARÍ", "GINEBRA", "EL CERRITO", "EL DOVIO", "ROLDANILLO", "LA UNION", "LA UNIÓN", "TORO", "OBANDO", "ULLOA", "VERSALLES", "ZARZAL", "CARTAGO", "ANSERMANUEVO", "ARGELIA", "BOLIVAR", "BOLÍVAR",
    "EL CAIRO", "EL AGUILA", "EL ÁGUILA", "ANDALUCIA", "ANDALUCÍA", "BUENAVENTURA"
]

KEYWORDS_OBRAS = [
    "OBRA", "OBRAS", "CONSTRUCCI", "MANTENIM", "MEJORAM", "INFRAESTRUC", "VIAL", "PUENTE", "CARRETER", "VIA", "VÍA", "PAVIMENT", "PLACAHUELLA", "CICLOV", "TRAMO",
    "ACUEDUCTO", "ALCANTARILL", "SANEAM", "DRENAJE", "COLEGIO", "ESCUELA", "EDUCAT", "HOSPIT", "SALUD", "PARQUE", "POLIDEPORT", "BIBLIOTECA", "CENTRO CULTURAL",
    "URBANIZ", "ANDEN", "ANDÉN", "MALECON", "MALECÓN", "TERMINAL", "REHABILIT", "RESTAUR", "REPARACI", "ADECUACI", "CONSERV"
]
KEYWORDS_INTERV = ["INTERVENTOR"]

obra_clauses = []
if CAMPO_TIPO:
    obra_clauses.append(f"upper({CAMPO_TIPO}) = upper('Obra')")
like_obras = []
like_interv = []
if CAMPO_OBJ:
    like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
    like_interv = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_INTERV]
    interv_and_obra = "( (" + " OR ".join(like_interv) + ") AND (" + " OR ".join(like_obras) + ") )"
    obras_only = "(" + " OR ".join(like_obras) + ")"
    obra_clauses.extend([interv_and_obra, obras_only])

if not obra_clauses:
    raise ValueError("No se encontraron campos para identificar 'obras'.")

if CAMPO_DEPTO:
    geo_clause = f"upper({CAMPO_DEPTO}) = upper('Valle del Cauca')"
elif CAMPO_MPIO:
    mpio_list = ", ".join(["'" + m.upper().replace("'", "''") + "'" for m in MUNICIPIOS_VALLE])
    geo_clause = f"upper({CAMPO_MPIO}) IN ({mpio_list})"
else:
    raise ValueError("No se encontró ni campo de Departamento ni de Municipio.")

WHERE = f"{geo_clause} AND (" + " OR ".join(obra_clauses) + ")"

assert "None" not in WHERE
assert any(tok in WHERE for tok in ("upper(", " like ", " IN "))

print("WHERE SoQL construido:", WHERE)

params_total = {"$select": "count(1)"}
res_total = requests.get(BASE_URL, params=params_total, headers=HEADERS, timeout=180)
res_total.raise_for_status()
total_dataset = int(res_total.json()[0]["count_1"]) if res_total.json() else 0

params_geo = {"$select": "count(1)", "$where": geo_clause}
res_geo = requests.get(BASE_URL, params=params_geo, headers=HEADERS, timeout=180)
res_geo.raise_for_status()
count_geo = int(res_geo.json()[0]["count_1"]) if res_geo.json() else 0

params_count = {"$select": "count(1)", "$where": WHERE}
res_count = requests.get(BASE_URL, params=params_count, headers=HEADERS, timeout=180)
res_count.raise_for_status()
count_val = int(res_count.json()[0]["count_1"]) if res_count.json() else 0

count_interv_geo = None
if CAMPO_OBJ and like_interv:
    params_interv = {"$select": "count(1)", "$where": geo_clause + " AND (" + " OR ".join(like_interv) + ")"}
    res_interv = requests.get(BASE_URL, params=params_interv, headers=HEADERS, timeout=180)
    res_interv.raise_for_status()
    count_interv_geo = int(res_interv.json()[0]["count_1"]) if res_interv.json() else 0

print("===== MINI REPORTE DE COBERTURA =====")
print(f"Total dataset (todas las filas): {total_dataset:,}")
print(f"Solo geografía (Valle del Cauca): {count_geo:,}")
print(f"Valle + Obras (incluye interventoría): {count_val:,}")
if count_interv_geo is not None:
    print(f"   └─ De los geográficos, con mención a INTERVENTOR*: {count_interv_geo:,}")
print("====================================")

def fetch_all(where_clause: str, batch: int = 50000) -> pd.DataFrame:
    frames = []
    offset = 0
    while True:
        params = {"$where": where_clause, "$limit": batch, "$offset": offset}
        r = requests.get(BASE_URL, params=params, headers=HEADERS, timeout=180)
        r.raise_for_status()
        data = r.json()
        if not data:
            break
        frames.append(pd.DataFrame(data))
        if len(data) < batch:
            break
        offset += batch
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

df_all = fetch_all(WHERE, batch=50000)

print("===== DESCARGA DEL FILTRO =====")
print(f"Filas recuperadas (Valle+Obras): {len(df_all):,}")
print(f"Conteo esperado por API (count_val): {count_val:,}")
if len(df_all) != count_val:
    print("Aviso: filas recuperadas difieren del conteo. Revisar límite/ofset o estabilidad del dataset.")
print("================================")

VAR_FECHA_INICIO    = "fecha_adjudicacion"
VAR_DURACION        = "duración"
VAR_UNIDAD_DURACION = "unidad_de_duracion"
VAR_PRECIO_BASE     = "precio_base"
VAR_VALOR_INICIAL   = "Valor Total Adjudicación"
VAR_ESTADO          = "estado_resumen"
VAR_ORDEN_ENTIDAD   = "ordenentidad"
VAR_MODALIDAD       = "modalidad_de_contratacion"

for col in [VAR_FECHA_INICIO, VAR_DURACION, VAR_UNIDAD_DURACION, VAR_PRECIO_BASE, VAR_VALOR_INICIAL, VAR_ESTADO, VAR_ORDEN_ENTIDAD, VAR_MODALIDAD]:
    if col not in df_all.columns:
        df_all[col] = pd.NA

df_all[VAR_FECHA_INICIO]  = pd.to_datetime(df_all[VAR_FECHA_INICIO], errors="coerce")
df_all[VAR_DURACION]      = pd.to_numeric(df_all[VAR_DURACION], errors="coerce")
df_all[VAR_PRECIO_BASE]   = pd.to_numeric(df_all[VAR_PRECIO_BASE], errors="coerce")
df_all[VAR_VALOR_INICIAL] = pd.to_numeric(df_all[VAR_VALOR_INICIAL], errors="coerce")

u = df_all[VAR_UNIDAD_DURACION].astype(str).str.lower().str.normalize('NFKD').str.encode('ascii', 'ignore').str.decode('ascii')
is_mes = u.str.contains("mes")
is_dia = u.str.contains("dia")
is_ano = u.str.contains("ano")

duracion_dias = np.where(is_dia, df_all[VAR_DURACION],
                   np.where(is_ano, df_all[VAR_DURACION] * 365,
                     np.where(is_mes, df_all[VAR_DURACION] * 30, df_all[VAR_DURACION] * 30)))
df_all["duracion_dias_calc"] = duracion_dias

df_all["fecha_fin_teorica"] = pd.to_datetime(df_all[VAR_FECHA_INICIO], errors="coerce") + pd.to_timedelta(df_all["duracion_dias_calc"], unit="D")

today = pd.Timestamp.today().normalize()
cond_valid = df_all[VAR_FECHA_INICIO].notna() & pd.notna(df_all["duracion_dias_calc"]) & df_all["fecha_fin_teorica"].notna()
df_all["terminada?"] = np.where(cond_valid & (today > df_all["fecha_fin_teorica"]), "sí, validar GENTE", "en ejecución")

if "adjudicado" in df_all.columns:
    adj = df_all["adjudicado"].astype(str).str.strip().str.lower()
    mask_adj = adj.isin(["si", "sí"])
    df_adj = df_all[mask_adj].copy()
else:
    df_adj = pd.DataFrame()

print("\n===== ESTADÍSTICA DESCRIPTIVA (adjudicado == Sí) – universo completo =====")
print("Registros adjudicados:", (len(df_adj) if not df_adj.empty else 0))

if not df_adj.empty:
    print("\nDuración (días) - describe():")
    print(df_adj["duracion_dias_calc"].describe())

    print("\nDistribución unidad_de_duracion:")
    print(df_adj[VAR_UNIDAD_DURACION].value_counts(dropna=False).head(20))

    if df_adj[VAR_PRECIO_BASE].notna().any():
        print("\nPrecio base - suma/promedio:")
        print("Suma:", float(df_adj[VAR_PRECIO_BASE].sum()))
        print("Promedio:", float(df_adj[VAR_PRECIO_BASE].mean()))

    if df_adj[VAR_VALOR_INICIAL].notna().any():
        print("\nValor Total Adjudicación - suma/promedio:")
        print("Suma:", float(df_adj[VAR_VALOR_INICIAL].sum()))
        print("Promedio:", float(df_adj[VAR_VALOR_INICIAL].mean()))

    print("\nEstado 'terminada?':")
    print(df_adj["terminada?"].value_counts(dropna=False))

    if CAMPO_MPIO and CAMPO_MPIO in df_adj.columns:
        print("\nTop municipios por conteo (adjudicados):")
        print(df_adj[CAMPO_MPIO].value_counts(dropna=False).head(15))

    if "entidad" in df_adj.columns:
        print("\nTop entidades por conteo (adjudicados):")
        print(df_adj["entidad"].value_counts(dropna=False).head(15))

    if VAR_MODALIDAD in df_adj.columns:
        print("\nModalidad de contratación (Top 15):")
        print(df_adj[VAR_MODALIDAD].value_counts(dropna=False).head(15))

    if VAR_ORDEN_ENTIDAD in df_adj.columns:
        print("\nOrden de la entidad:")
        print(df_adj[VAR_ORDEN_ENTIDAD].value_counts(dropna=False))
else:
    print("No existe la columna 'adjudicado' o no hay registros 'Sí' para reportar.")
print("============================================================================")


WHERE SoQL construido: upper(departamento_entidad) = upper('Valle del Cauca') AND (upper(tipo_de_contrato) = upper('Obra'))
===== MINI REPORTE DE COBERTURA =====
Total dataset (todas las filas): 7,548,652
Solo geografía (Valle del Cauca): 598,536
Valle + Obras (incluye interventoría): 6,815
===== DESCARGA DEL FILTRO =====
Filas recuperadas (Valle+Obras): 6,815
Conteo esperado por API (count_val): 6,815

===== ESTADÍSTICA DESCRIPTIVA (adjudicado == Sí) – universo completo =====
Registros adjudicados: 2306

Duración (días) - describe():
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: duracion_dias_calc, dtype: float64

Distribución unidad_de_duracion:
unidad_de_duracion
día(s)       1264
Mes(es)      1038
Semana(s)       3
Año(s)          1
Name: count, dtype: int64

Precio base - suma/promedio:
Suma: 6036452946753.0
Promedio: 2617715935.2788377

Estado 'terminada?':
terminada?
en ejecución    2306
Name: count, dtype: int64

T

In [ ]:
import pandas as pd
import re
import unicodedata

if 'df_all' in locals() and isinstance(df_all, pd.DataFrame) and not df_all.empty:
    _df = df_all.copy()
elif 'df_preview' in locals() and isinstance(df_preview, pd.DataFrame) and not df_preview.empty:
    _df = df_preview.copy()
else:
    raise SystemExit('No hay DataFrame disponible (df_all/df_preview) para analizar entidades.')

cand_ent_cols = [
    'entidad','entidad_contratante','nombre_entidad','nombre_de_la_entidad',
    'entidad_contratista','nombre_de_la_entidad_estatal','entidad_estatal'
]
col_entidad = next((c for c in cand_ent_cols if c in _df.columns), None)
if not col_entidad:
    raise SystemExit('No se encontró la columna de entidad en el DataFrame.')

VAR_VALOR_INICIAL = 'Valor Total Adjudicación'
if VAR_VALOR_INICIAL not in _df.columns:
    _df[VAR_VALOR_INICIAL] = pd.NA

if 'terminada?' not in _df.columns:
    _df['terminada?'] = pd.NA

if 'duracion_dias_calc' not in _df.columns:
    _df['duracion_dias_calc'] = pd.NA

def _norm(s: str) -> str:
    if not isinstance(s, str):
        s = ''
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')
    return s.lower().strip()

ent_norm = _df[col_entidad].astype(str).map(_norm)
mask_gob = ent_norm.str.contains(r'\bgobernacion\b') & ent_norm.str.contains(r'\bvalle\b')
mask_sec = ent_norm.str.contains(r'\bsecretar', regex=True) & ent_norm.str.contains(r'\bvalle\b')

assert _norm('GOBERNACIÓN DEL VALLE DEL CAUCA').find('gobernacion') >= 0
assert bool(re.search(r'\bgobernacion\b', _norm('Gobernación del Valle'))) is True
assert bool(re.search(r'\bsecretar', _norm('Secretaría de Infraestructura del Valle del Cauca'))) is True
assert bool(re.search(r'\bvalle\b', _norm('Secretaría de Salud del Valle del Cauca'))) is True

print('===== ENTIDADES (Top 20) =====')
print(_df[col_entidad].value_counts(dropna=False).head(50))
print('===============================')

gob_count = int(mask_gob.sum())
sec_count = int(mask_sec.sum())
print(f'Total registros Gobernacion del Valle: {gob_count:,}')
print(f'Total registros Secretarias del Valle: {sec_count:,}')

if 'ordenentidad' in _df.columns:
    ord_counts = _df['ordenentidad'].astype(str).map(_norm).value_counts(dropna=False)
    print('\n===== ordenentidad =====')
    print(ord_counts.head(100))
    print('DEPARTAMENTAL:', int(ord_counts.get('departamental', 0)))

_df_gs = _df[mask_gob | mask_sec].copy()
print('\n===== SUBREPORTE: Gobernacion + Secretarias =====')
print('Registros:', len(_df_gs))
if len(_df_gs):
    if 'terminada?' in _df_gs.columns:
        print("Estado 'terminada?':")
        print(_df_gs['terminada?'].value_counts(dropna=False))
    if 'duracion_dias_calc' in _df_gs.columns:
        try:
            print('\nDuracion (dias) – describe():')
            print(pd.to_numeric(_df_gs['duracion_dias_calc'], errors='coerce').describe())
        except Exception:
            pass
    if VAR_VALOR_INICIAL in _df_gs.columns:
        vals = pd.to_numeric(_df_gs[VAR_VALOR_INICIAL], errors='coerce')
        if vals.notna().any():
            print('\nValor Total Adjudicacion – suma/promedio:')
            print('Suma:', float(vals.sum()))
            print('Promedio:', float(vals.mean()))
else:
    print('Sin coincidencias para Gobernacion/Secretarias con los criterios actuales.')


===== ENTIDADES (Top 20) =====
entidad
SECRETARIA DE EDUCACION DE CALI                                                                               404
SANTIAGO DE CALI DISTRITO ESPECIAL - SECRETARIA DEL DEPORTE Y LA RECREACIÓN                                   370
SANTIAGO DE CALI DISTRITO ESPECIAL - UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PUBLICOS                     267
SANTIAGO DE CALI DISTRITO ESPECIAL - SECRETARIA DE INFRAESTRUCTURA                                            256
SOCIEDAD DE ACUEDUCTOS Y ALCANTARILLADOS DEL VALLE DEL CAUCA S.A. - E.S.P.                                    251
ALCALDIA MUNICIPAL DE BUGA                                                                                    197
ALCALDÍA MUNICIPAL DE ROLDANILLO                                                                              178
CVC                                                                                                           175
ALCALDÍA MUNICIPIO DE PALMIRA                    